In [23]:
import numpy as np
import subprocess
import os
import glob 


# Make sure you use a working directory that you want your output files stored in
# the defautl output folder is 'MDOutputFiles' which will be generated in your working directory
#
wdir = ''
output_folder = f'{wdir}/MDOutputFiles'
os.makedirs(output_folder, exist_ok = True)


#
# if you're not using smog files you can change the path for .mdp files to be used and comment out the next block of code
#
#
#
used = glob.glob('.used')
sysnm = ""
with open(used, 'r') as f:
    lines = f.readlines()
    for line in lines:
        if line.startswith('level of graining') and line[18:26] == 'All-Atom':
            sim_type = 'aa'
        elif line.startswith('level of graining') and line[18:26] == 'CA':
            sim_type = 'ca'

# The preferred mdp settings for these coars graining simulations are generated via the smog server 
#  https://smog-server.org/prepare_a_simulation.html, 
# 
#
#
#
#
#
aa_mdp = '.mdp'
ca_mdp = '.mdp'

# Temperature range will dictate the start and stopping points for what .tpr files will be generated
# dt chooses the temeprature step in kelvin between each .tpr file
#
#
trng = 80, 120
lower_temp, upper_temp = map(int, trng.split())

dt = 10

gro = glob.glob(f'{wdir}/*.gro')
top = glob.glob(f'{wdir}/*.top')

gro = gro[0]
top = top[0]

if sim_type == 'aa':
    with open(aa_mdp, 'r') as f:
        lines = f.readlines()
        
    ref_t_found = False
    gen_temp_found = False

    for i in range(lower_temp, upper_temp+1, dt):
        if os.path.exists(f'{output_folder}/{sysnm}_{i}_aa.tpr'):
            continue
        new_lines = []
        for line in lines:
            if line.startswith('ref_t'):
                new_line = f'ref_t = {i}\n'
                ref_t_found = True
            elif line.startswith('gen_temp'):
                new_line = f'gen_temp = {i}\n'
                gen_temp_found = True
            else:
                new_line = line
            new_lines.append(new_line)

        if not ref_t_found:
            new_lines.append(f'ref_t = {i}\n')
        if not gen_temp_found:
            new_lines.append(f'gen_temp = {i}\n')

        with open(aa_mdp, 'w') as f:
            f.writelines(new_lines)  
        # generate tprs 
        
        tpr_pro = subprocess.run(f"gmx grompp -f {aa_mdp} -c {gro} -r {gro} -p {top} -o {output_folder}/{sysnm}_{i}_aa -maxwarn 1", shell=True)
    tprfiles = glob.glob(f'{output_folder}/*.tpr')

    for file in tprfiles:
        if os.path.exists(f'{output_folder}/{i}_aa.xtc'):
            continue
        else:
            process = subprocess.run([f'gmx mdrun -deffnm {sysnm}_{i}_aa -s {file}; done'], shell=True, cwd=output_folder)
if sim_type == 'ca':
    with open(ca_mdp, 'r') as f:
        lines = f.readlines()

    ref_t_found = False
    gen_temp_found = False

    for i in range(lower_temp, upper_temp+1, dt):
        if os.path.exists(f'{output_folder}/{sysnm}_{i}_ca.tpr'):
            continue
        new_lines = []
        for line in lines:
            if line.startswith('ref_t'):
                new_line = f'ref_t = {i}\n'
                ref_t_found = True
            elif line.startswith('gen_temp'):
                new_line = f'gen_temp = {i}\n'
                gen_temp_found = True
            else:
                new_line = line
            new_lines.append(new_line)

        if not ref_t_found:
            new_lines.append(f'ref_t = {i}\n')
        if not gen_temp_found:
            new_lines.append(f'gen_temp = {i}\n')

        with open(ca_mdp, 'w') as f:
            f.writelines(new_lines)   
        # generate tprs 
        tpr_pro = subprocess.run(f"gmx grompp -f {aa_mdp} -c {gro} -r {gro} -p {top} -o {output_folder}/{sysnm}_{i}_ca.tpr -maxwarn 1", shell=True)                                   
    tprfiles = glob.glob(f'{output_folder}/*.tpr')

#This is the point in which the gromacs will start running the generated simulations if you don't want to run the files comment out the next block
#
#
    for file in tprfiles:
        if os.path.exists(f'{output_folder}/{i}_aa.xtc'):
            continue
        else:
            process = subprocess.run([f'gmx mdrun -deffnm {sysnm}_{i}_aa -s {file}; done'], shell=True, cwd=output_folder)

                  

TypeError: unhashable type: 'list'